In [28]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

Eğer importlar ve kütüphaneler çalışmıyorsa, gerekli kütüphaneleri indirin. Aşağıda vereceğim satırı yeni bir hücre oluşturup çalıştırın. Daha sonra Jupyter'deki bulunduğunuz kerneli yeniden başlatın.

pip install webdriver-manager

pip install pymupdf pandas selenium openpyxl

tek bir seferde çalıştırmanız yeterli. Tek seferde çalıştırıp başarılı bir şekilde kütüphaneleri kurduysanız hücreyi silip kerneli yeniden başlatıp bu kodu çalıştırabilirsiniz.

In [29]:
def extract_courses():
    print(f"Yazdırılan excel dersleri:")
    ceng_courses = set()
    excel_path = "dersler.xlsx"


    df = pd.read_excel(excel_path)
    
    for index, row in df.iterrows():

        if 'CEC' in str(row['DERS']):#-----------------------------------------------!
            course_code = str(row['DERS']).strip()
            if(course_code[3] != " "):
                course_code = course_code[:3] + " " + course_code[3:]
            course_code = course_code.lower()
            print(course_code)

            if row['CENG'] == 'X': #-----------------------------------------------!
                ceng_courses.add(course_code)


    print("Bölümünüz için uygun dersler:")
    print(ceng_courses)
    
    return ceng_courses if ceng_courses else set()


In [30]:
def get_courses_from_web():
    url = "https://www.cankaya.edu.tr/dersler/"
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    time.sleep(2)
    
    try:

        dropdown = driver.find_element(By.NAME, "DersCode")
        
        dropdown.send_keys("CEC") #-----------------------------------------------!
        
        dropdown.send_keys(Keys.RETURN)
        time.sleep(3)
        
        print(f"Yazdırılan Webdeki dersler")
        courses = set()
        table_rows = driver.find_elements(By.XPATH, "//tr[@role='row']")
        for row in table_rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            
            if cols and "CEC" in cols[0].text: #-----------------------------------------------!
                
                course_code = cols[0].text.replace("\u00a0", "").strip().lower()  # Özel boşluk karakterlerini düzelt ve normalleştir
                print(course_code)
                courses.add(course_code)
        
        driver.quit()
        return courses if courses else set()  # Boş set dönerek None hatasını önle
    except Exception as e:
        print(f"Web scraping sırasında hata oluştu: {e}")
        driver.quit()
        return set()



Uyarı: Bu blok, gerekli excel dosyasını indirmeniz gerektiğini ve excel dosyası ile bu python kodunun aynı klasör ve/ya dizinde olmasını gerektiğini hatırlatır. Ayrıca, çankaya web sitesindeki HTML yapısı zamanla değişebilir. O yüzden, aramak istediğiniz ögeyi kodu düzenleyip yazmanız gerekir, ya da kod tamamen geçersiz olabilir. 

Ayrıca, istediğiniz dersin size uygun olup olmadığını görmek için kod içerisinde şu düzenlemeyi yapmanız lazım;

Adım 1

            if 'CEC' in str(row['DERS']): <-------------- ' ' bunun gibi tırnak arasına istediğiniz dersin Kod ismini yazın.
            
            (CEC, MAN, ALM, THEA gibi)

Adım 2

            dropdown.send_keys("CEC")<-------------------------- tırnak arasına istediğiniz dersin Kod ismini yazın.

Adım 3

    if cols and "CEC" in cols[0].text: <-------------------------  tırnak arasına istediğiniz dersin Kod ismini yazın.

Eğer bölümünüz de farklı ise;


Adım 1

      if row['CENG'] == 'X': <-------------------------  tırnak arasına bölümünüzün kodunu yazın. (CENG, SENG, ME gibi)
      
Kod içerisinde değiştireceğiniz yerler #-----------------------------------------------! şeklinde işaretlenmiştir.
                        

In [31]:
courses = extract_courses()



Yazdırılan excel dersleri:
cec 103
cec 104
cec 105
cec 107
cec 108
cec 114
cec 121
cec 205
cec 210
cec 211
cec 212
cec 214
cec 216
cec 217
cec 218
cec 222
cec 224
cec 226
cec 228
cec 301
cec 307
Bölümünüz için uygun dersler:
{'cec 210', 'cec 218', 'cec 103', 'cec 121', 'cec 307', 'cec 205', 'cec 214', 'cec 216', 'cec 107', 'cec 301', 'cec 228', 'cec 222', 'cec 224', 'cec 212', 'cec 217', 'cec 226', 'cec 104', 'cec 105'}


In [32]:
web_courses = get_courses_from_web()


if not courses:
    print("PDF'den hiç ders bulunamadı!")
if not web_courses:
    print("Webte hiç ders bulunamadı!")

common_courses = courses.intersection(web_courses)

print("\n Alabileceğin seçmeli dersleri:")
for course in common_courses:
    print(course)

Yazdırılan Webdeki dersler
cec 104
cec 108
cec 109
cec 111
cec 112
cec 113
cec 116
cec 203
cec 210
cec 211
cec 212
cec 214
cec 216
cec 217
cec 224
cec 226
cec 234

 Alabileceğin seçmeli dersleri:
cec 210
cec 214
cec 216
cec 226
cec 224
cec 212
cec 217
cec 104
